In [75]:
#Load Data & Extract Time-Based Features

In [76]:
import pandas as pd
import numpy as np

# Load CitiBike trip data
citibike_df = pd.read_csv("/Users/shashankhmg/Documents/AXA-Casestudy/Data-Science-Challenge/data/processed/cleaned_citibike_data.csv")  

/var/folders/tw/yc01l_99601d14ks8nxwg1dr0000gn/T/ipykernel_30447/2193106995.py:5: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  citibike_df = pd.read_csv("/Users/shashankhmg/Documents/AXA-Casestudy/Data-Science-Challenge/data/processed/cleaned_citibike_data.csv")


In [77]:
citibike_df.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='object')

In [78]:
def clean_start_station_id(df):
    print('Shape of df before cleaning',df.shape)
    def convert_to_float(value):
        try:
            return float(value)  # Convert to float
        except ValueError:
            #print(f"Invalid start_station_id found: {value}")  # Print problematic value
            return None  # Mark for deletion

    # Apply the function to convert values, marking invalid ones as None
    df["start_station_id"] = df["start_station_id"].apply(convert_to_float)

    # Drop rows where conversion failed
    df = df.dropna(subset=["start_station_id"]).reset_index(drop=True)
    print('Shape of df after cleaning',df.shape)

    return df

citibike_df = clean_start_station_id(citibike_df)


Shape of df before cleaning (2119681, 13)
Shape of df after cleaning (2119462, 13)


In [79]:
# Convert timestamps
citibike_df["started_at"] = pd.to_datetime(citibike_df["started_at"])
citibike_df["hour_of_day"] = citibike_df["started_at"].dt.hour
citibike_df["day_of_week"] = citibike_df["started_at"].dt.dayofweek
citibike_df["weekend"] = (citibike_df["day_of_week"] >= 5).astype(int)  # 1 if Sat/Sun

# Extract month and rush hour feature
citibike_df["month"] = citibike_df["started_at"].dt.month
citibike_df["rush_hour"] = ((citibike_df["hour_of_day"].between(7, 9)) | 
                            (citibike_df["hour_of_day"].between(16, 19))).astype(int)

In [80]:
citibike_df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,hour_of_day,day_of_week,weekend,month,rush_hour
0,56BD148A05E26915,electric_bike,2025-01-01 22:19:06.324,2025-01-01 22:23:32.899,W 36 St & 7 Ave,6483.06,W 24 St & 7 Ave,6257.03,40.752149,-73.989539,40.744876,-73.995299,member,22,2,0,1,0
1,700CCCDF00C08077,electric_bike,2025-01-09 15:20:43.991,2025-01-09 15:26:19.137,Broadway & E 19 St,5980.11,W 24 St & 7 Ave,6257.03,40.738290,-73.990060,40.744876,-73.995299,member,15,3,0,1,0
2,B1A3FAFA5FE651CE,electric_bike,2025-01-01 12:54:26.571,2025-01-01 13:03:03.565,Jackson Ave & 46 Rd,6203.02,Crescent St & Broadway,6827.11,40.745248,-73.947333,40.763359,-73.928647,member,12,2,0,1,0
3,9CB0F28054BA5EBC,electric_bike,2025-01-14 13:43:59.151,2025-01-14 13:49:04.543,Broadway & E 19 St,5980.11,W 24 St & 7 Ave,6257.03,40.738290,-73.990060,40.744876,-73.995299,member,13,1,0,1,0
4,A1B8FDD4FAF0C2B2,electric_bike,2025-01-13 07:20:37.176,2025-01-13 07:29:18.015,Willis Ave & E 143 St,7798.02,1 Ave & E 110 St,7522.02,40.812299,-73.920370,40.792327,-73.938300,member,7,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2119457,0EDA53461D0388D5,classic_bike,2025-01-24 14:46:32.047,2025-01-24 14:53:27.119,34 Ave & 31 St,6720.07,31 Ave & Steinway St,6782.01,40.759604,-73.927144,40.761149,-73.917007,member,14,4,0,1,0
2119458,C30264245638AA98,electric_bike,2025-01-29 20:39:37.864,2025-01-29 20:43:38.814,Driggs Ave & S 2 St,5235.10,Bayard St & Leonard St,5442.05,40.712628,-73.960575,40.719156,-73.948854,member,20,2,0,1,0
2119459,E7A2D0EC9B5197E5,electric_bike,2025-01-21 15:46:38.904,2025-01-21 15:49:19.936,W 11 St & 6 Ave,5914.03,W 10 St & Washington St,5847.06,40.735324,-73.998004,40.733424,-74.008515,casual,15,1,0,1,0
2119460,5FF2284D69F9D931,electric_bike,2025-01-19 09:21:26.100,2025-01-19 09:31:36.489,Pleasant Ave & E 116 St,7540.05,E 80 St & 2 Ave,7121.02,40.794990,-73.933330,40.773914,-73.954395,member,9,6,1,1,1


In [81]:
citibike_df.shape

(2119462, 18)

In [82]:
#Aggregate Data by Station & Time

In [83]:
# Group by station and time to get trip count
demand_data = citibike_df.groupby(["start_station_id", "hour_of_day", "day_of_week"]).size().reset_index(name="trip_count")
demand_data

,start_station_id,hour_of_day,day_of_week,trip_count
0,2733.03,4,6,1
1,2733.03,7,0,1
2,2733.03,7,1,4
3,2733.03,7,2,5
4,2733.03,7,3,2
...,...,...,...,...
241729,8897.05,18,4,1
241730,8897.05,20,4,1
241731,8897.05,22,2,2
241732,8897.05,22,3,2


In [84]:
# Merge time-based features
demand_data = demand_data.merge(citibike_df[["start_station_id", "hour_of_day", "day_of_week", "weekend", "month", "rush_hour"]],
                                on=["start_station_id", "hour_of_day", "day_of_week"], 
                                how="left")


In [85]:
demand_data

,start_station_id,hour_of_day,day_of_week,trip_count,weekend,month,rush_hour
0,2733.03,4,6,1,1,1,0
1,2733.03,7,0,1,0,1,1
2,2733.03,7,1,4,0,1,1
3,2733.03,7,1,4,0,1,1
4,2733.03,7,1,4,0,1,1
...,...,...,...,...,...,...,...
2119457,8897.05,22,2,2,0,1,0
2119458,8897.05,22,2,2,0,1,0
2119459,8897.05,22,3,2,0,1,0
2119460,8897.05,22,3,2,0,1,0


In [86]:
# Compute Historical Trends (Rolling Averages)
# ie. we compute the avg number of trips in the past 7 & 30 days 
#Rolling 7-day average (rolling_avg_7days): This computes the average number of rides that started at each station over the past 7 days.
#Rolling 30-day average (rolling_avg_30days): This computes the average number of rides that started at each station over the past 30 days.
#7-day average - Captures weekly fluctuations (e.g., weekend vs. weekday demand shifts).
#30-day average - Captures long-term trends, accounting for seasonal variations.


In [87]:
# Ensure datetime format
citibike_df["started_at"] = pd.to_datetime(citibike_df["started_at"])

# Sort values before rolling operation
citibike_df = citibike_df.sort_values(["start_station_id", "started_at"])

# Set 'started_at' as index for rolling calculations
citibike_df.set_index("started_at", inplace=True)

# Compute rolling averages for each station
citibike_df["rolling_avg_7days"] = (
    citibike_df.groupby("start_station_id")["start_station_id"]
    .rolling("7D")
    .count()
    .reset_index(level=0, drop=True)
)

citibike_df["rolling_avg_30days"] = (
    citibike_df.groupby("start_station_id")["start_station_id"]
    .rolling("30D")
    .count()
    .reset_index(level=0, drop=True)
)

# Reset index after rolling operation
citibike_df.reset_index(inplace=True)

In [88]:
citibike_df.shape

(2119462, 20)

In [89]:
citibike_df['rolling_avg_30days'].sort_values(ascending=False)

1178754    8815.0
1178753    8814.0
1178752    8813.0
1178748    8813.0
1178747    8813.0
            ...  
258010        1.0
2084089       1.0
257375        1.0
2084230       1.0
0             1.0
Name: rolling_avg_30days, Length: 2119462, dtype: float64

In [90]:
citibike_df['rolling_avg_7days'].sort_values(ascending=False)

1178608    2433.0
1178611    2433.0
1178607    2433.0
1178606    2432.0
1178610    2432.0
            ...  
1191415       1.0
1196715       1.0
1196853       1.0
64822         1.0
0             1.0
Name: rolling_avg_7days, Length: 2119462, dtype: float64

In [91]:
# Compute station-level average for rolling 7-day and 30-day windows
station_avg_rolling = citibike_df.groupby("start_station_id")[["rolling_avg_7days", "rolling_avg_30days"]].mean().reset_index()

# Rename columns for clarity
station_avg_rolling.rename(columns={
    "rolling_avg_7days": "avg_rolling_7days",
    "rolling_avg_30days": "avg_rolling_30days"
}, inplace=True)
station_avg_rolling

,start_station_id,avg_rolling_7days,avg_rolling_30days
0,2733.03,13.290323,31.500000
1,2782.02,23.154545,55.500000
2,2821.05,29.727891,73.938776
3,2832.03,30.837500,80.406250
4,2861.02,23.165138,54.908257
...,...,...,...
2141,8866.01,5.105263,10.000000
2142,8876.04,9.044444,23.000000
2143,8879.02,23.285714,52.961905
2144,8879.03,36.120219,91.912568


In [92]:
citibike_df["start_station_id"].nunique()


2146

In [93]:
# Merge with demand data
demand_data = demand_data.merge(station_avg_rolling[["start_station_id", "avg_rolling_7days", "avg_rolling_30days"]],
                                on="start_station_id", how="left")


In [94]:
demand_data

,start_station_id,hour_of_day,day_of_week,trip_count,weekend,month,rush_hour,avg_rolling_7days,avg_rolling_30days
0,2733.03,4,6,1,1,1,0,13.290323,31.5
1,2733.03,7,0,1,0,1,1,13.290323,31.5
2,2733.03,7,1,4,0,1,1,13.290323,31.5
3,2733.03,7,1,4,0,1,1,13.290323,31.5
4,2733.03,7,1,4,0,1,1,13.290323,31.5
...,...,...,...,...,...,...,...,...,...
2119457,8897.05,22,2,2,0,1,0,11.636364,28.0
2119458,8897.05,22,2,2,0,1,0,11.636364,28.0
2119459,8897.05,22,3,2,0,1,0,11.636364,28.0
2119460,8897.05,22,3,2,0,1,0,11.636364,28.0


In [95]:
#Add Location-Based Features

# Select relevant columns and drop duplicates to ensure one entry per station
# Drop duplicates, keeping the first occurrence of each station ID because it looks like some stations have more than one lat and long LOL. so i choose to ignore it 
station_location_df = citibike_df.drop_duplicates(subset=["start_station_id"], keep="first")[["start_station_id", "start_lat", "start_lng"]]


In [96]:
station_location_df

,start_station_id,start_lat,start_lng
0,2733.03,40.633385,-74.016562
62,2782.02,40.635679,-74.020005
172,2821.05,40.635560,-74.012980
319,2832.03,40.637033,-74.022141
479,2861.02,40.638566,-74.010147
...,...,...,...
2119055,8866.01,40.883240,-73.905870
2119074,8876.04,40.884920,-73.890840
2119119,8879.02,40.886300,-73.899700
2119224,8879.03,40.885240,-73.900770


In [97]:
# Merge geolocation data
demand_data = demand_data.merge(
    station_location_df[["start_station_id", "start_lat", "start_lng"]].drop_duplicates(),
    on="start_station_id",
    how="left"
)


In [98]:
demand_data

,start_station_id,hour_of_day,day_of_week,trip_count,weekend,month,rush_hour,avg_rolling_7days,avg_rolling_30days,start_lat,start_lng
0,2733.03,4,6,1,1,1,0,13.290323,31.5,40.633385,-74.016562
1,2733.03,7,0,1,0,1,1,13.290323,31.5,40.633385,-74.016562
2,2733.03,7,1,4,0,1,1,13.290323,31.5,40.633385,-74.016562
3,2733.03,7,1,4,0,1,1,13.290323,31.5,40.633385,-74.016562
4,2733.03,7,1,4,0,1,1,13.290323,31.5,40.633385,-74.016562
...,...,...,...,...,...,...,...,...,...,...,...
2119457,8897.05,22,2,2,0,1,0,11.636364,28.0,40.886020,-73.902320
2119458,8897.05,22,2,2,0,1,0,11.636364,28.0,40.886020,-73.902320
2119459,8897.05,22,3,2,0,1,0,11.636364,28.0,40.886020,-73.902320
2119460,8897.05,22,3,2,0,1,0,11.636364,28.0,40.886020,-73.902320


In [100]:
demand_data.isnull().sum()

start_station_id      0
hour_of_day           0
day_of_week           0
trip_count            0
weekend               0
month                 0
rush_hour             0
avg_rolling_7days     0
avg_rolling_30days    0
start_lat             0
start_lng             0
dtype: int64

In [101]:
demand_data.to_csv('/Users/shashankhmg/Documents/AXA-Casestudy/Data-Science-Challenge/data/model_data/demand_forecast_model/demand_data.csv')

In [103]:
# Merge geolocation data
demand_data = demand_data.merge(
    citibike_df[["start_station_id", "start_station_name"]].drop_duplicates(),
    on="start_station_id",
    how="left"
)


In [104]:
demand_data

,start_station_id,hour_of_day,day_of_week,trip_count,weekend,month,rush_hour,avg_rolling_7days,avg_rolling_30days,start_lat,start_lng,start_station_name
0,2733.03,4,6,1,1,1,0,13.290323,31.5,40.633385,-74.016562,67 St & Erik Pl
1,2733.03,7,0,1,0,1,1,13.290323,31.5,40.633385,-74.016562,67 St & Erik Pl
2,2733.03,7,1,4,0,1,1,13.290323,31.5,40.633385,-74.016562,67 St & Erik Pl
3,2733.03,7,1,4,0,1,1,13.290323,31.5,40.633385,-74.016562,67 St & Erik Pl
4,2733.03,7,1,4,0,1,1,13.290323,31.5,40.633385,-74.016562,67 St & Erik Pl
...,...,...,...,...,...,...,...,...,...,...,...,...
2121600,8897.05,22,2,2,0,1,0,11.636364,28.0,40.886020,-73.902320,W 238 St & Tibbett Ave
2121601,8897.05,22,2,2,0,1,0,11.636364,28.0,40.886020,-73.902320,W 238 St & Tibbett Ave
2121602,8897.05,22,3,2,0,1,0,11.636364,28.0,40.886020,-73.902320,W 238 St & Tibbett Ave
2121603,8897.05,22,3,2,0,1,0,11.636364,28.0,40.886020,-73.902320,W 238 St & Tibbett Ave


In [105]:
import json
# Create a dictionary mapping station names to all relevant info
station_dict = {}
for _, row in demand_data.iterrows():
    station_dict[row["start_station_name"]] = {
        "start_station_id": row["start_station_id"],
        "start_lat": row["start_lat"],
        "start_lng": row["start_lng"],
        "avg_rolling_7days": row["avg_rolling_7days"],
        "avg_rolling_30days": row["avg_rolling_30days"]
    }

# Save to JSON
with open("station_data_2.json", "w") as f:
    json.dump(station_dict, f, indent=4)

